# Libraries

In [1]:
from bs4 import BeautifulSoup
import re
import pandas as pd
import csv
import requests

In [2]:
with open('ApartmentsData.csv','w',newline='',encoding='utf-8-sig',) as f:
    writer = csv.writer(f)
    writer.writerow(['Location','Details','Space','Bedrooms','Bathrooms','Description','Price'])
    page_no=1
    while page_no != 97:
        res = requests.get(f'https://eg.opensooq.com/ar/%D8%A7%D9%84%D9%82%D8%A7%D9%87%D8%B1%D8%A9/%D8%B9%D9%82%D8%A7%D8%B1%D8%A7%D8%AA-%D9%84%D9%84%D8%A8%D9%8A%D8%B9/%D8%B4%D9%82%D9%82-%D9%84%D9%84%D8%A8%D9%8A%D8%B9?search=true&page={page_no}')
        soap = BeautifulSoup(res.text,'html.parser')
        main_div = soap.find_all('div',attrs={'class':
                               'sc-cfbb9bd4-3 dQelXJ postDet flex-1 flex flexSpaceBetween flexDirectionColumn'}
                 )
        for i in range(len(main_div)):
            #Location & Description
            location = main_div[i].find('span').text
            descreption = main_div[i].find('h2').text
            
            #Space(m^2)
            try:
                space = int(soap.find_all('div',attrs={'class':
                           'sc-cfbb9bd4-3 dQelXJ postDet flex-1 flex flexSpaceBetween flexDirectionColumn'}
             )[i].find_all('div')[2].text.split(',')[2].split()[0])
            except:
                space = 0
            #Bedrooms
            try:
                bedrooms = int(soap.find_all('div',attrs={'class':
                           'sc-cfbb9bd4-3 dQelXJ postDet flex-1 flex flexSpaceBetween flexDirectionColumn'}
             )[i].find_all('div')[2].text.split(',')[0].split()[0])
            except:
                bedrooms = 1
            #Bathrooms
            try:
                l = soap.find_all('div',attrs={'class':
                           'sc-cfbb9bd4-3 dQelXJ postDet flex-1 flex flexSpaceBetween flexDirectionColumn'}
             )[i].find_all('div')[2].text.split(',')[1].split()
            
                if len(l)==3:
                    bathrooms = int(l[0])
                else:
                    if l[0]=='حمّامين':
                        bathrooms = 2
                    else:
                        bathrooms = 1
            except:
                bathrooms = 1
            #Details
            
            det=soap.find_all('div',attrs={'class':
                           'sc-cfbb9bd4-3 dQelXJ postDet flex-1 flex flexSpaceBetween flexDirectionColumn'}
             )[i].find_all('div')[2].text.split(',')[:-1]
            
            details = ''.join(det).replace('\u200f','|').strip('|')
            
            #Price
            try:
                price_str = re.findall('\d+',soap.find_all('div',attrs={'class':
                           'sc-cfbb9bd4-3 dQelXJ postDet flex-1 flex flexSpaceBetween flexDirectionColumn'}
             )[i].find('div',attrs={'class':'postPrice ms-auto bold alignSelfCenter font-19'}).text)
                price = int(''.join(price_str))
            except:
                price = 0
            writer.writerow([location,descreption,space,bedrooms,bathrooms,details,price])
        page_no+=1

In [3]:
df = pd.read_csv('ApartmentsData.csv')
df

,Location,Details,Space,Bedrooms,Bathrooms,Description,Price
0,القطامية,دوبلكس للبيع 180 متر مدينه النخيل بالم سيتي عل...,180,4,3,٤ غرف نوم | ٣ حمّامات,2800000
1,التجمع الخامس,امتلك شقتك بي سعر الطرح بتقسيط علي 10 سنوات,129,2,2,٢ غرفتا نوم | حمّامين,20000
2,المعادي,شقه سوبر لوكس بسعر رائع قبل ارتفاع الاسعار,110,3,2,٣ غرف نوم | حمّامين,1450000
3,أخرى,مدينه الفردوس,220,3,3,٣ غرف نوم | ٣ حمّامات,2500000
4,مدينة نصر,500 شقه للبيع للاستثماى,250,3,2,٣ غرف نوم | حمّامين,0
...,...,...,...,...,...,...,...
2828,القاهرة,شقه للبيع B15,100,2,2,٢ غرفتا نوم | حمّامين,350000
2829,القاهرة,شقه للبيع بمدينه نصر,200,3,2,٣ غرف نوم | حمّامين,2200000
2830,القاهرة,شقة 200م في كمبوند عايش وساكن اول سكن,200,3,3,٣ غرف نوم | ٣ حمّامات,6110000
2831,القاهرة,شقق للبيع في مدينتي,200,3,2,٣ غرف نوم | حمّامين,5600000
